In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np
import os
from torchvision.datasets import ImageFolder
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
from sklearn.metrics import accuracy_score
from itertools import product
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
from sklearn.neighbors import KNeighborsClassifier 



In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.models import resnet18
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
from torchvision.models import resnet18
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report

from PIL import Image
classes= ['MUS','NORM', 'STR']
# Load the trained model
model_path = '/kaggle/input/savedmodel/model_final.pth'  # Update this path
model = resnet18(pretrained=False)  # Use the same model architecture as you used for training
model.fc = nn.Linear(model.fc.in_features, len(classes))  # Adjust the fully connected layer as per your model
# Load the state dictionary from the file
state_dict = torch.load(model_path)

# Filter out unnecessary keys
filtered_state_dict = {k: v for k, v in state_dict.items() if 'total_ops' not in k and 'total_params' not in k}

# Load the filtered state dictionary into the model
model.load_state_dict(filtered_state_dict, strict=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

# Load and preprocess the sample image
def preprocess_image(image_path):
    # Define the transformations. Adjust as per your requirement.
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Example size, adjust to match training
        transforms.ToTensor(),
        # Add any other transformations you used during training (e.g., normalization)
    ])

    # Load image
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

# Example image path
sample_image_path = '/kaggle/input/images/colon_cancer_GettyImages1214015886_Header.webp'  # Update this path
sample_tensor = preprocess_image(sample_image_path).to(device)

# Forward pass to get the prediction
with torch.no_grad():
    output = model(sample_tensor)
    probabilities = torch.nn.functional.softmax(output, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1)

# Get the class name from the predicted class index
predicted_class_name = classes[predicted_class.item()]
print(f'Predicted Class: {predicted_class_name}')
print(f'Class Probabilities: {probabilities}')


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Predicted Class: NORM
Class Probabilities: tensor([[3.0336e-29, 1.0000e+00, 7.8893e-43]], device='cuda:0')
